## Tacotron 2 inference code
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Tacotron2_Sounds/tacotron2


/content/drive/MyDrive/Tacotron2_Sounds/tacotron2


In [4]:
!ls


audio_processing.py  filelists	       loss_scaler.py	  stft.py
data_utils.py	     hparams.py        model.py		  tacotron2
demo.wav	     inference.ipynb   multiproc.py	  tensorboard.png
denoiser.py	     layers.py	       plotting_utils.py  text
denoiser.py.1	     LICENSE	       __pycache__	  train.py
distributed.py	     logger.py	       README.md	  utils.py
Dockerfile	     loss_function.py  requirements.txt   waveglow


In [5]:
!pip install Unidecode


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 18.7 MB/s eta 0:00:00


In [6]:
!wget https://raw.githubusercontent.com/NVIDIA/waveglow/master/denoiser.py -P /content/drive/MyDrive/Tacotron2_Sounds/tacotron2/



--2025-05-18 20:48:31--  https://raw.githubusercontent.com/NVIDIA/waveglow/master/denoiser.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1605 (1.6K) [text/plain]
Saving to: ‘/content/drive/MyDrive/Tacotron2_Sounds/tacotron2/denoiser.py.2’

denoiser.py.2       100%[===================>]   1.57K  --.-KB/s    in 0.001s  

2025-05-18 20:48:32 (2.30 MB/s) - ‘/content/drive/MyDrive/Tacotron2_Sounds/tacotron2/denoiser.py.2’ saved [1605/1605]



In [7]:
from denoiser import Denoiser


#### Import libraries and setup matplotlib

In [8]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch


from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from denoiser import Denoiser

In [9]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom',
                       interpolation='none')

In [10]:
!ls /content/drive/MyDrive/Tacotron2_Sounds/tacotron2/distributed.py


/content/drive/MyDrive/Tacotron2_Sounds/tacotron2/distributed.py


In [11]:
from hparams import create_hparams
hparams = create_hparams()
hparams.sampling_rate = 22050

In [12]:
import sys
sys.path.append('/content/drive/MyDrive/Tacotron2_Sounds/tacotron2')


In [13]:
from tacotron2.distributed import apply_gradient_allreduce



In [14]:
from train import load_model


#### Setup hparams

In [15]:
hparams = create_hparams()
hparams.sampling_rate = 22050

#### Load model from checkpoint

In [32]:
checkpoint_path = "/content/drive/MyDrive/Tacotron2_Sounds/tacotron2/tacotron2_statedict.pt"


In [34]:
!ls /content/drive/MyDrive/Tacotron2_Sounds/


100.wav  115.wav  20.wav  35.wav  4.wav   64.wav  79.wav  93.wav
101.wav  116.wav  21.wav  36.wav  50.wav  65.wav  7.wav   94.wav
102.wav  117.wav  22.wav  37.wav  51.wav  66.wav  80.wav  95.wav
103.wav  118.wav  23.wav  38.wav  52.wav  67.wav  81.wav  96.wav
104.wav  119.wav  24.wav  39.wav  53.wav  68.wav  82.wav  97.wav
105.wav  11.wav   25.wav  3.wav   54.wav  69.wav  83.wav  98.wav
106.wav  120.wav  26.wav  40.wav  55.wav  6.wav   84.wav  99.wav
107.wav  12.wav   27.wav  41.wav  56.wav  70.wav  85.wav  9.wav
108.wav  13.wav   28.wav  42.wav  57.wav  71.wav  86.wav  metadata.csv
109.wav  14.wav   29.wav  43.wav  58.wav  72.wav  87.wav  outdir
10.wav	 15.wav   2.wav   44.wav  59.wav  73.wav  88.wav  tacotron2
110.wav  16.wav   30.wav  45.wav  5.wav   74.wav  89.wav  train_corrected.txt
111.wav  17.wav   31.wav  46.wav  60.wav  75.wav  8.wav   train_fullpath.txt
112.wav  18.wav   32.wav  47.wav  61.wav  76.wav  90.wav  train.txt
113.wav  19.wav   33.wav  48.wav  62.wav  77.wav  91.wa

In [26]:
import os
print(os.path.exists(checkpoint_path))


False


In [29]:
checkpoint_path = "tacotron2_statedict.pt"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval().half()

FileNotFoundError: [Errno 2] No such file or directory: 'tacotron2_statedict.pt'

#### Load WaveGlow for mel2audio synthesis and denoiser

In [ ]:
waveglow_path = 'waveglow_256channels.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

#### Prepare text input

In [ ]:
text = "Waveglow is really awesome!"
sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()

#### Decode text input and plot results

In [ ]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))

#### Synthesize audio from spectrogram using WaveGlow

In [ ]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

#### (Optional) Remove WaveGlow bias

In [ ]:
audio_denoised = denoiser(audio, strength=0.01)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate)